In [1]:
import h5py
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from statsmodels.tsa.arima.model import ARIMA
import math
import numpy.linalg as la
from tqdm import tqdm
from tabulate import tabulate

# Загрузка данных
with h5py.File('../data/raw_data/METR-LA/METR-LA.h5', 'r') as file:
    axis0 = file['df']['axis0'][:]              # Идентификаторы датчиков
    axis1 = file['df']['axis1'][:]              # Метки времени
    timestamps = pd.to_datetime(axis1)          # Преобразование меток времени в формат datetime
    df_data = file['df']['block0_values'][:]    # Данные замеров скорости

axis0 = [x.decode('utf-8') for x in axis0]
metr_la = pd.DataFrame(df_data, index=timestamps, columns=axis0)
metr_la = metr_la.iloc[:2016, :2]

# Загрузка матрицы смежности
with open('../data/raw_data/METR-LA/adj_METR-LA.pkl', 'rb') as file:
    data = pickle.load(file, encoding='bytes')

node_ids = [x.decode('utf-8') for x in data[0]]  # Получаем список id узлов из data[0]
adj_matrix = data[2]  # Получаем матрицу смежности из data[2]
metr_la_adj = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids)  # Создание DataFrame

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tabulate import tabulate

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler

# Нормализация данных
def normalize_data(trainX, testX):
    scaler = MinMaxScaler()
    trainX_shape = trainX.shape
    testX_shape = testX.shape
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX_shape[-1])).reshape(trainX_shape)
    testX = scaler.transform(testX.reshape(-1, testX_shape[-1])).reshape(testX_shape)
    return trainX, testX, scaler

# Функция для добавления дополнительных фичей
def add_features(data, **kwargs):
    """
    Добавляет фичи в данные, поддерживая канал измерений.

    Args:
        data (np.ndarray): Исходные данные, shape: [time, nodes, channels] или [time, nodes].
        **kwargs: Словарь функций {feature_name: function}, где функция применяется по времени для каждого узла.

    Returns:
        np.ndarray: Обновленные данные с дополнительными фичами, shape: [time, nodes, channels+N].
    """
    # Если данных нет размерности каналов, добавляем её
    if len(data.shape) == 2:  # [time, nodes]
        data = data[..., np.newaxis]  # Преобразуем в [time, nodes, 1]
    
    # Преобразуем данные в DataFrame для удобства добавления фичей
    time, nodes, channels = data.shape
    df = pd.DataFrame(data.reshape(time, -1))

    # Добавляем фичи
    new_features = []
    for feature_name, func in kwargs.items():
        feature = df.apply(func, axis=0).fillna(0).values  # Заполняем NaN значениями 0
        new_features.append(feature[..., np.newaxis])  # Добавляем размерность каналов
    
    # Объединяем оригинальные данные и новые фичи
    new_features = np.concatenate(new_features, axis=-1)  # [time, nodes, N]
    data = np.concatenate([data, new_features], axis=-1)  # [time, nodes, channels + N]
    return data

# Предобработка данных
def preprocess_data_multichannel(data, time_len, rate, seq_len, pre_len):
    """
    Преобразует данные в формат [batch, seq_len, nodes, channels].

    Args:
        data (np.ndarray): Исходные данные, shape: [time, nodes, channels].
        time_len (int): Общее количество временных точек.
        rate (float): Доля обучающей выборки.
        seq_len (int): Длина входной последовательности.
        pre_len (int): Длина предсказания.

    Returns:
        tuple: (trainX, trainY, testX, testY)
    """
    train_size = int(time_len * rate)
    train_data = data[:train_size]
    test_data = data[train_size:time_len]

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0: seq_len])  # Shape: [seq_len, nodes, channels]
        trainY.append(a[seq_len: seq_len + pre_len])  # Shape: [pre_len, nodes, channels]
    for i in range(len(test_data) - seq_len - pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0: seq_len])
        testY.append(b[seq_len: seq_len + pre_len])
    
    return np.array(trainX), np.array(trainY), np.array(testX), np.array(testY)

# Пример использования
time_len = len(metr_la)
rate = 0.8
seq_len = 12
pre_len = 3

# Добавление mean и std
metr_la_array = metr_la.values  # Исходные данные [time, nodes]
metr_la_extended = add_features(
    metr_la_array,
    mean=lambda x: pd.Series(x).rolling(window=5, min_periods=1).mean().values,
    std=lambda x: pd.Series(x).rolling(window=5, min_periods=1).std().values
)

# Преобразование в массив [time, nodes, channels]
trainX, trainY, testX, testY = preprocess_data_multichannel(metr_la_extended, time_len, rate, seq_len, pre_len)

# Нормализация данных
trainX_norm, testX_norm, scaler_X = normalize_data(trainX, testX)
trainY_norm, testY_norm, scaler_Y = normalize_data(trainY, testY)

# Преобразование в тензоры PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainX_norm = torch.tensor(trainX_norm, dtype=torch.float32).to(device)
trainY_norm = torch.tensor(trainY_norm, dtype=torch.float32).to(device)
testX_norm = torch.tensor(testX_norm, dtype=torch.float32).to(device)
testY_norm = torch.tensor(testY_norm, dtype=torch.float32).to(device)

# Вывод размерности данных
print(f"TrainX shape: {trainX_norm.shape}")  # [batch, seq_len, nodes, channels]
print(f"TrainY shape: {trainY_norm.shape}")  # [batch, pre_len, nodes, channels]
print(f"TestX shape: {testX_norm.shape}")    # [batch, seq_len, nodes, channels]
print(f"TestY shape: {testY_norm.shape}")    # [batch, pre_len, nodes, channels]


TrainX shape: torch.Size([1597, 12, 2, 3])
TrainY shape: torch.Size([1597, 3, 2, 3])
TestX shape: torch.Size([389, 12, 2, 3])
TestY shape: torch.Size([389, 3, 2, 3])


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, pre_len=3):
        super(LSTMModel, self).__init__()
        self.pre_len = pre_len
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: [batch, seq_len, nodes, channels]
        batch, seq_len, nodes, channels = x.size()
        x = x.view(batch * nodes, seq_len, channels)  # Merge batch and nodes dimensions for LSTM
        
        h_lstm, _ = self.lstm(x)  # h_lstm shape: [batch * nodes, seq_len, hidden_size]
        h_lstm = h_lstm[:, -self.pre_len:, :]  # Take last `pre_len` steps
        
        out = self.fc(h_lstm)  # shape: [batch * nodes, pre_len, output_size]
        out = out.view(batch, nodes, self.pre_len, -1).permute(0, 2, 1, 3)  # [batch, pre_len, nodes, output_size]
        return out  # Final shape: [batch, pre_len, nodes, 1]


In [48]:
def train_model(model, trainX, trainY, testX, testY, scaler_Y, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(trainX)  # outputs shape: [batch, pre_len, output_size]
        loss = criterion(outputs, trainY)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # Оценка модели
    model.eval()
    with torch.no_grad():
        predictions_norm = model(testX).cpu().numpy()  # predictions shape: [batch, pre_len, output_size]

    # Восстановление данных
    predictions = scaler_Y.inverse_transform(predictions_norm.reshape(-1, predictions_norm.shape[-1]))
    predictions = predictions.reshape(predictions_norm.shape)  # Восстановление исходной формы
    actuals = scaler_Y.inverse_transform(testY.cpu().numpy().reshape(-1, testY.shape[-1]))
    actuals = actuals.reshape(testY.shape)  # Восстановление исходной формы

    # Метрики
    rmse = np.sqrt(mean_squared_error(actuals.reshape(-1), predictions.reshape(-1)))
    mae = mean_absolute_error(actuals.reshape(-1), predictions.reshape(-1))
    r2 = r2_score(actuals.reshape(-1), predictions.reshape(-1))
    var = np.var(actuals - predictions)
    return rmse, mae, r2, var

In [49]:
from sklearn.model_selection import ParameterGrid
from tabulate import tabulate

# Определение сетки параметров для LSTM
param_grid = {
    "hidden_size": [32, 64, 128],
    "num_layers": [4],
    "lr": [0.002],
    "epochs": [100, 300]
}

# Функция для Grid Search
def grid_search_lstm(model_class, param_grid, trainX, trainY, testX, testY, scaler_Y):
    best_rmse = float("inf")
    best_params = None
    results = []

    # Перебор всех комбинаций параметров
    for params in ParameterGrid(param_grid):
        print(f"Testing LSTM with params: {params}")

        # Создание модели LSTM
        model = model_class(
            input_size=trainX.shape[3],  # Размерность входных фичей (channels)
            hidden_size=params["hidden_size"],
            output_size=trainY.shape[3],  # Размерность выходных фичей (channels)
            num_layers=params["num_layers"],
            pre_len=trainY.shape[1]
        )

        # Обучение модели
        rmse, mae, r2, var = train_model(
            model, trainX, trainY, testX, testY, scaler_Y,
            epochs=params["epochs"], lr=params["lr"]
        )

        # Сохранение результатов
        results.append({
            "params": params,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "var": var
        })

        # Обновление лучших параметров
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params

    return best_params, best_rmse, results

# Выполнение Grid Search для LSTM
best_params, best_rmse, results = grid_search_lstm(
    LSTMModel, param_grid, trainX_norm, trainY_norm, testX_norm, testY_norm, scaler_Y
)

# Вывод лучших параметров и результатов
print(f"Best params for LSTM: {best_params}")
print(f"Best RMSE for LSTM: {best_rmse}")
print("All results:")
print(tabulate(results, headers="keys"))


Testing LSTM with params: {'epochs': 100, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}
Epoch [100/100], Loss: 0.0215
Testing LSTM with params: {'epochs': 100, 'hidden_size': 64, 'lr': 0.002, 'num_layers': 4}
Epoch [100/100], Loss: 0.0211
Testing LSTM with params: {'epochs': 100, 'hidden_size': 128, 'lr': 0.002, 'num_layers': 4}
Epoch [100/100], Loss: 0.0215
Testing LSTM with params: {'epochs': 300, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}
Epoch [100/300], Loss: 0.0213
Epoch [200/300], Loss: 0.0196
Epoch [300/300], Loss: 0.0178
Testing LSTM with params: {'epochs': 300, 'hidden_size': 64, 'lr': 0.002, 'num_layers': 4}
Epoch [100/300], Loss: 0.0214
Epoch [200/300], Loss: 0.0187
Epoch [300/300], Loss: 0.0173
Testing LSTM with params: {'epochs': 300, 'hidden_size': 128, 'lr': 0.002, 'num_layers': 4}
Epoch [100/300], Loss: 0.0209
Epoch [200/300], Loss: 0.0178
Epoch [300/300], Loss: 0.0170
Best params for LSTM: {'epochs': 300, 'hidden_size': 64, 'lr': 0.002, 'num_layers': 4}
Best 